In [1]:
# importando bibliotecas necessárias
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import requests

In [2]:
# fazer uma requisição GET da url da página 
# esse método retorna um objeto response que checamos se retornou o status 200 (bem sucedida)
# status_code: checa se a requisção foi bem sucedida | content: passamos o html da página
req = requests.get('https://www.basketball-reference.com/leagues/NBA_2018_totals.html')

In [3]:
if req.status_code == 200:
    print('Requisição bem sucedida!')
    content = req.content

Requisição bem sucedida!


In [4]:
# parseando o html com o Beautiful Soup
soup = BeautifulSoup(content, 'html.parser')

In [7]:
# procurando com método find() o objeto que queremos extrais (tabela de estatisticas)
table = soup.find(name='table')

In [9]:
# carregando os dados em um df do pandas
table_str = str(table)
df = pd.read_html(table_str)[0]

#### E se tiver mais de uma tabela na página?
É preciso especificar qual table é usando atributo attrs

table = soup.find(name='table', attrs={'id':'confs_standings_W'})

In [10]:
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Álex Abrines,SG,24,OKC,75,8,1134,115,291,...,.848,26,88,114,28,38,8,25,124,353
1,2,Quincy Acy,PF,27,BRK,70,8,1359,130,365,...,.817,40,217,257,57,33,29,60,149,411
2,3,Steven Adams,C,24,OKC,76,76,2487,448,712,...,.559,384,301,685,88,92,78,128,215,1056
3,4,Bam Adebayo,C,20,MIA,69,19,1368,174,340,...,.721,118,263,381,101,32,41,66,138,477
4,5,Arron Afflalo,SG,32,ORL,53,3,682,65,162,...,.846,4,62,66,30,4,9,21,56,179


In [11]:
# limpando os dados 
drop_indexes = df[df['Rk'] == 'Rk'].index # Pega indexes onde a coluna 'Rk' possui valor 'Rk'
df.drop(drop_indexes, inplace=True) # elimina os valores dos index passados da tabela

In [12]:
# converter os valores numéricos da tabela porque estão sendo lidos como objetos
# utilizar o método pd.to_numeric
numeric_cols = df.columns.drop(['Player','Pos','Tm'])
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)

In [13]:
# importando bibliotecas para visualização em gráficos
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
# agrupando os dados por jogador e somando os valores
grouped_df = df.groupby('Player', as_index=False).sum()
sorted_df = grouped_df.sort_values(by=['PTS'],axis=0,ascending=False)

In [22]:
sorted_df.head()

,Player,Rk,Age,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
484,Tobias Harris,600,75,160,160,5336,1124,2442,1.384,366,...,2.475,116,758,874,384,142,70,210,328,2972
153,Eric Bledsoe,156,84,148,148,4644,936,1978,1.349,250,...,2.376,102,466,568,740,298,84,432,370,2626
41,Blake Griffin,573,84,116,116,3940,872,1992,1.312,222,...,2.354,146,710,856,668,82,36,330,278,2484
324,LeBron James,248,33,82,82,3026,857,1580,0.542,149,...,0.731,97,612,709,747,116,71,347,136,2251
266,Jordan Clarkson,294,75,162,4,3776,860,1908,1.355,224,...,2.405,114,320,434,442,116,12,254,214,2248
